In [1]:
import sys
import os
root_dir = os.path.join(os.getcwd(), '..')
sys.path.append(root_dir)

from pathlib import Path
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from utilities import create_model
from utilities import TrainerConfiguration
from utilities import TrainDataset

2022-07-06 20:23:29.726133: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-06 20:23:29.737124: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-06 20:23:29.737807: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-06 20:23:29.739622: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [2]:
BATCH_SIZE=128
KERNEL_SIZE=4
ACTIVATION_FUNCTION='relu'
LEARNING_RATE=0.001
NUM_UNITS_DENSE=256
NUM_UNITS_LTSM2=1024
### run 55 parameters
DROPOUT=0.12489316869910207
NUM_UNITS_LTSM1=512
### run 41 parameters
# DROPOUT=0.1
# NUM_UNITS_LTSM1=768

In [3]:
model_path = 'gs://iam-model-staging/run_55_train/model/'
pred_model = keras.models.load_model(model_path)

pred_model.save_weights(Path('checkpoints', 'pred'))

In [4]:
full_model = create_model(KERNEL_SIZE, ACTIVATION_FUNCTION, NUM_UNITS_DENSE, DROPOUT, NUM_UNITS_LTSM1, NUM_UNITS_LTSM2, LEARNING_RATE)
full_model.load_weights(Path('checkpoints', 'pred'))
for i in range(len(full_model.layers)):
    full_model.layers[i].trainable=False
full_model.layers[6].trainable = True
full_model.layers[-1].trainable = True
full_model.compile(keras.optimizers.Adam(learning_rate=LEARNING_RATE))

Model: "ocr_model_v1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 image (InputLayer)             [(None, 400, 100, 1  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 400, 100, 32  544         ['image[0][0]']                  
                                )                                                                 
                                                                                                  
 pool1 (MaxPooling2D)           (None, 200, 50, 32)  0           ['Conv1[0][0]']                  
                                                                                       

In [ ]:
# load Steyermark or Standley images
folder = Path('images/standley_4058_train')
metadata = Path('words_metadata.csv')
data = TrainDataset()
data.create_dataset(128, folder, metadata)

# fit open layers
history = full_model.fit(data.train_dataset, epochs=50, validation_data=data.validation_dataset)

full_model.save('retrained.model')

prediction_model = tf.keras.models.Model(
    full_model.get_layer(name='image').input, full_model.get_layer(name='dense_layer').output
)
prediction_model.compile(tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE))
prediction_model.save('retrained-prediction.model')

In [ ]:
# fine tune
full_model.trainable = True
full_model.compile(keras.optimizers.Adam(learning_rate=1e-5))
history_fine_tune = full_model.fit(data.train_dataset, epochs=25, validation_data=data.validation_dataset)


full_model.save('fine_tuned.model')

prediction_model = tf.keras.models.Model(
    full_model.get_layer(name='image').input, full_model.get_layer(name='dense_layer').output
)
prediction_model.compile(tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE))
prediction_model.save('fine_tuned-prediction.model')